# Caso 3

1. Un portafolio de inversión consiste de 1 millón de dólares de facial colocados en la nota
del tesoro con CUSIP 91282CGU9 y de 1.5 millones en la nota con CUSIP 91282CGS4.
La descripción de los instrumentos se brinda en los documentos adjuntos. Note que
las notas del tesoro pagan intereses semestralmente. Suponga que la fecha de análisis
es el día de la emisión de los instrumentos, o sea el 31 de marzo de 2023.
Se adjunta, además, un histórico de curvas de rendimiento cero cupón calculadas por
el Banco de la Reserva Federal.



In [237]:
!pip install openpyxl
!pip install QuantLib-Python

In [238]:
# Carga de librerías
import pandas as pd
import numpy as np
from scipy import stats
import QuantLib as ql
import re

In [239]:
# Carga de datos

curvas_rendimientos = pd.read_excel('/content/CurvaActualizadaCaso3.xlsx')

# Mostrar las primeras filas del DataFrame
print(curvas_rendimientos.head())

        Date   plazo_1  plazo_30  plazo_90  plazo_180  plazo_360  plazo_720  \
0 2019-03-15  0.026136  0.026013  0.025775   0.025454   0.024933   0.024279   
1 2019-03-18  0.025970  0.025867  0.025666   0.025395   0.024953   0.024398   
2 2019-03-19  0.025896  0.025803  0.025622   0.025377   0.024974   0.024465   
3 2019-03-20  0.026120  0.025973  0.025687   0.025301   0.024671   0.023867   
4 2019-03-21  0.025953  0.025820  0.025561   0.025210   0.024634   0.023891   

   plazo_1080  plazo_1800  plazo_2520  plazo_3600  
0    0.024009    0.024164    0.024773    0.025913  
1    0.024173    0.024336    0.024919    0.026020  
2    0.024257    0.024418    0.024993    0.026101  
3    0.023515    0.023625    0.024267    0.025492  
4    0.023558    0.023652    0.024257    0.025431  


In [240]:
# Variables globales

delta = 0.0001

periodicidad = 2
# Tasas
tasa_cupon_1 = 0.03875
tasa_cupon_2 = 0.03625

# Precios
precio_1 = 99.849511
precio_2 = 99.993866



In [241]:

#Fechas
fecha_emision = pd.to_datetime('2023-03-31')
fecha_vencimiento = pd.to_datetime('2025-03-31')
fecha_vencimiento_2 = pd.to_datetime('2030-03-31')


In [242]:
fecha_calc_ql  = fecha_emision

fecha_calc_ql = ql.Date(fecha_calc_ql.day, fecha_calc_ql.month, fecha_calc_ql.year)

ql.Settings.instance().evaluationDate = fecha_calc_ql

In [243]:
qlCalendar = ql.BespokeCalendar('Costa Rica')
qlCalendar.addWeekend(ql.Sunday)
qlCalendar.addWeekend(ql.Saturday)

for year in range(2023, 2031):
    qlCalendar.addHoliday(ql.Date( 1, 1, year)) # Año Nuevo
    qlCalendar.addHoliday(ql.Date( 1, 5, year))  # Día del trabajador
    qlCalendar.addHoliday(ql.Date(25, 7, year)) # Anexión de Guanacaste
    qlCalendar.addHoliday(ql.Date( 2, 8, year))  # Día de la Virgen
    qlCalendar.addHoliday(ql.Date(15, 9, year)) # Independencia
    qlCalendar.addHoliday(ql.Date(25,12, year)) # Navidad

qlCalendar.addHoliday(ql.Date( 1, 1, 2025)) # Año Nuevo
qlCalendar.addHoliday(ql.Date( 6, 4, 2023)) # Jueves Santo 2023
qlCalendar.addHoliday(ql.Date(11, 4, 2024)) # Jueves Santo
qlCalendar.addHoliday(ql.Date( 7, 4, 2023)) # Viernes Santo 2023
qlCalendar.addHoliday(ql.Date(12, 4, 2024)) # Viernes Santo



In [244]:
fecha_vence_ql_1 = ql.Date(fecha_vencimiento.day, fecha_vencimiento.month, fecha_vencimiento.year)
fecha_vence_ql_2 = ql.Date(fecha_vencimiento_2.day, fecha_vencimiento_2.month, fecha_vencimiento_2.year)

fecha_emision_ql = ql.Date(fecha_emision.day, fecha_emision.month, fecha_emision.year)


In [245]:
qlTenor = ql.Period(periodicidad)
qlConvencion = ql.Following
dateGeneration = ql.DateGeneration.Backward
monthEnd = False

In [246]:
cronograma_bono_1 = ql.Schedule(fecha_emision_ql,fecha_vence_ql_1,qlTenor,
                         qlCalendar,qlConvencion,qlConvencion,dateGeneration,monthEnd)
# list(cronograma)
pd.DataFrame({'date':list(cronograma_bono_1)})

,date
0,"March 31st, 2023"
1,"October 2nd, 2023"
2,"April 1st, 2024"
3,"September 30th, 2024"
4,"March 31st, 2025"


In [247]:
cronograma_bono_2 = ql.Schedule(fecha_emision_ql,fecha_vence_ql_2,qlTenor,
                         qlCalendar,qlConvencion,qlConvencion,dateGeneration,monthEnd)
# list(cronograma)
pd.DataFrame({'date':list(cronograma_bono_2)})

,date
0,"March 31st, 2023"
1,"October 2nd, 2023"
2,"April 1st, 2024"
3,"September 30th, 2024"
4,"March 31st, 2025"
5,"September 30th, 2025"
6,"March 31st, 2026"
7,"September 30th, 2026"
8,"March 31st, 2027"
9,"September 30th, 2027"


In [248]:
# Definición del día de conteo utilizando el método 30/360
dayCount = ql.Thirty360(ql.Thirty360.BondBasis)

# Tipo de capitalización
compound_type = ql.Compounded

# Frecuencia de capitalización
frequency = ql.Semiannual

# Días de liquidación
settlementDays = 0



## a) Analice el riesgo del portafolio por medio de un enfoque paramétrico, usando
una aproximación delta normal.

###  Creacion de los bonos

In [249]:
bono_1 = ql.FixedRateBond(settlementDays,100,cronograma_bono_1,[tasa_cupon_1],dayCount)
bono_2 = ql.FixedRateBond(settlementDays,100,cronograma_bono_2,[tasa_cupon_2],dayCount)

In [250]:
flujos_1 = bono_1.cashflows()
fecha = [item.date() for item in flujos_1]
monto = [item.amount() for item in flujos_1]
print(pd.DataFrame({'Fechas':fecha,'Montos':monto}))

                 Fechas      Montos
0     October 2nd, 2023    1.959028
1       April 1st, 2024    1.926736
2  September 30th, 2024    1.926736
3      March 31st, 2025    1.937500
4      March 31st, 2025  100.000000


In [251]:
flujos_2 = bono_2.cashflows()
fecha = [item.date() for item in flujos_2]
monto = [item.amount() for item in flujos_2]
print(pd.DataFrame({'Fechas':fecha,'Montos':monto}))

                  Fechas      Montos
0      October 2nd, 2023    1.832639
1        April 1st, 2024    1.802431
2   September 30th, 2024    1.802431
3       March 31st, 2025    1.812500
4   September 30th, 2025    1.812500
5       March 31st, 2026    1.812500
6   September 30th, 2026    1.812500
7       March 31st, 2027    1.812500
8   September 30th, 2027    1.812500
9       March 31st, 2028    1.812500
10     October 2nd, 2028    1.832639
11       April 2nd, 2029    1.812500
12     October 1st, 2029    1.802431
13       April 1st, 2030    1.812500
14       April 1st, 2030  100.000000


### Cálculo de rendimeintos

In [252]:
(rendimiento_1 := bono_1.bondYield(precio_1,dayCount,compound_type,frequency) )

0.03953995412919653

In [253]:
(rendimiento_2 := bono_2.bondYield(precio_2,dayCount,compound_type,frequency) )

0.036259854614233596

### Intereses

In [254]:
interes_rendimiento_1 = ql.InterestRate(rendimiento_1,dayCount,compound_type,frequency)

In [255]:
interes_rendimiento_2 = ql.InterestRate(rendimiento_2,dayCount,compound_type,frequency)

### Duración

In [256]:
(duracion_MC_1 := ql.BondFunctions.duration(bono_1,interes_rendimiento_1,ql.Duration.Macaulay) )

1.9436542005940463

In [257]:
(duracion_MC_2 := ql.BondFunctions.duration(bono_2,interes_rendimiento_2,ql.Duration.Macaulay) )

6.247126663327023

In [258]:
(duracion_mod_1 := ql.BondFunctions.duration(bono_1,interes_rendimiento_1,ql.Duration.Modified) )

1.905973155033298

In [259]:
(duracion_mod_2 := ql.BondFunctions.duration(bono_2,interes_rendimiento_2,ql.Duration.Modified) )

6.135883540767966

### Curvas

In [260]:
curvas_rendimientos['Date'] = pd.to_datetime(curvas_rendimientos['Date'])
curvas_rendimientos.set_index('Date', inplace=True)



In [261]:
curvas_rendimientos_actual = curvas_rendimientos.iloc[-1,:]
tasa_actual = curvas_rendimientos_actual.values

In [262]:
curvas_rendimientos

,plazo_1,plazo_30,plazo_90,plazo_180,plazo_360,plazo_720,plazo_1080,plazo_1800,plazo_2520,plazo_3600
Date,,,,,,,,,,
2019-03-15,0.026136,0.026013,0.025775,0.025454,0.024933,0.024279,0.024009,0.024164,0.024773,0.025913
2019-03-18,0.025970,0.025867,0.025666,0.025395,0.024953,0.024398,0.024173,0.024336,0.024919,0.026020
2019-03-19,0.025896,0.025803,0.025622,0.025377,0.024974,0.024465,0.024257,0.024418,0.024993,0.026101
2019-03-20,0.026120,0.025973,0.025687,0.025301,0.024671,0.023867,0.023515,0.023625,0.024267,0.025492
2019-03-21,0.025953,0.025820,0.025561,0.025210,0.024634,0.023891,0.023558,0.023652,0.024257,0.025431
...,...,...,...,...,...,...,...,...,...,...
2023-03-27,0.051852,0.051032,0.049470,0.047430,0.044223,0.040162,0.037918,0.035945,0.035446,0.035742
2023-03-28,0.051151,0.050487,0.049202,0.047483,0.044665,0.040826,0.038517,0.036308,0.035663,0.035865
2023-03-29,0.051196,0.050565,0.049341,0.047690,0.044957,0.041168,0.038847,0.036591,0.035912,0.036077


### Plazos

In [263]:
plazos = curvas_rendimientos_actual.index.values
plazos_2 = [int(re.search(r'\d+', plazo).group()) for plazo in plazos]
plazos_limpios = np.array(plazos_2)
plazos_limpios

array([   1,   30,   90,  180,  360,  720, 1080, 1800, 2520, 3600])

In [264]:
tasa_actual

array([0.05343767, 0.05259337, 0.05097032, 0.04881758, 0.04535153,
       0.04079181, 0.03817697, 0.03579613, 0.03510458, 0.03521748])

In [265]:
plazos_limpios_ql= fecha_calc_ql + plazos_limpios
plazos_limpios_ql = ql.DateVector(list(plazos_limpios_ql))


In [266]:
curva_ql0 = ql.ZeroCurve(plazos_limpios_ql, tasa_actual, ql.Thirty360(ql.Thirty360.BondBasis))
curva_ql = ql.YieldTermStructureHandle(curva_ql0)

In [267]:
estructura_plazos = ql.ZeroSpreadedTermStructure(
        curva_ql,ql.QuoteHandle(ql.SimpleQuote(0)), ql.Compounded,
        ql.Semiannual,dayCount)

In [268]:
pricing_engine = ql.DiscountingBondEngine(ql.RelinkableYieldTermStructureHandle(estructura_plazos))

In [269]:
bono_1.setPricingEngine(pricing_engine)
bono_1.NPV()

99.52623809960488

### Calculo del precio


In [270]:
def Calculadora_Precio(spread, bono):
    estructura_plazos = ql.ZeroSpreadedTermStructure(
            curva_ql,ql.QuoteHandle(ql.SimpleQuote(spread)), ql.Compounded,
            ql.Semiannual,dayCount)
    pricing_engine = ql.DiscountingBondEngine(
            ql.RelinkableYieldTermStructureHandle(estructura_plazos))

    bono.setPricingEngine(pricing_engine)
    return bono.NPV()

In [271]:
Calculadora_Precio(.01, bono_1)

97.65368772591926

In [272]:
def ErrorPrecio(spread, precio, bono):
    return precio-Precio(spread, bono)

In [273]:
from scipy.optimize import bisect

In [274]:
spread_estimado_1 = bisect(lambda spread: ErrorPrecio(spread, precio_1, bono_1), -1, 1)
spread_estimado_1

-0.0017021180010488024

In [275]:
spread_estimado_2 = bisect(lambda spread: ErrorPrecio(spread, precio_2, bono_2), -1, 1)
spread_estimado_2

0.0006169575153762707

In [276]:
Calculadora_Precio(spread_estimado_1, bono_1)

99.8495110002618

In [277]:
Calculadora_Precio(spread_estimado_2, bono_2)

99.99386600013406

### Cálculo del riesgo

In [278]:
from scipy.stats import norm

In [279]:
nivel_conf = 0.999
zq = norm.ppf(nivel_conf)
cambios_tasas = curvas_rendimientos.diff().iloc[1:]
sigma_plazo = cambios_tasas.std()
vol = sigma_plazo.iloc[9]
vol

0.0006080116336959643

In [280]:
N1 = 1000000
N2 = 1500000
M1 = N1*precio_1
M2 = N2*precio_2
V = M1 + M2         #Valor de la cartera
(VaR := M1*duracion_mod_1*zq*vol+M2*duracion_mod_2*zq*vol)

2086771.919431425

## b) Mejore el análisis de riesgo usando las curvas adjuntas y un enfoque por componentes principales.


In [281]:
from sklearn import decomposition

In [282]:
pca = decomposition.PCA(n_components=3, whiten = False)

In [283]:
cambios_en_tasas = np.diff(curvas_rendimientos.values,axis = 0)

In [284]:
x_r = pca.fit_transform(cambios_en_tasas)

In [285]:
comp = pca.components_

In [286]:
print(pca.explained_variance_ratio_)

[0.97524959 0.01454019 0.00824121]


In [304]:
def PriceTermStructShift(spread,shift,bono):
    ql_curvacero_shift = ql.ZeroCurve(fecha_calc_ql+plazos_limpios,tasa_actual+shift,ql.Thirty360(ql.Thirty360.BondBasis))
    ql_curva_shift = ql.YieldTermStructureHandle(ql_curvacero_shift)
    term_structure_spread = ql.ZeroSpreadedTermStructure(ql_curva_shift,ql.QuoteHandle(ql.SimpleQuote(spread)),ql.Compounded,ql.Semiannual,dayCount)
    pricing_engine_shift = ql.DiscountingBondEngine(ql.RelinkableYieldTermStructureHandle(term_structure_spread))
    bono.setPricingEngine(pricing_engine_shift)
    return bono.NPV()

In [308]:
dur_pca_1 = np.zeros(3)
dur_pca_2 = np.zeros(3)
delta = .0001
for icomp in range(0,3):
    nuevo_precio = PriceTermStructShift(spread_estimado_1,delta*comp[icomp], bono_1)
    dur_pca_1[icomp] = -(1/precio_1)*(precio_1-nuevo_precio)/(delta)
    nuevo_precio = PriceTermStructShift(spread_estimado_2,delta*comp[icomp], bono_2)
    dur_pca_2[icomp] = -(1/precio_2)*(precio_2-nuevo_precio)/(delta)

In [309]:
print(dur_pca_1)
print(dur_pca_2)

[ 0.00388182 -0.7908985   0.03265895]
[ 0.03464589 -2.66280454  0.22351953]


In [300]:
from numpy.linalg import norm

In [301]:
covarianza = np.cov(np.transpose(x_r))
covarianza

array([[ 1.24052678e-04,  3.77729777e-21, -9.97337310e-21],
       [ 3.77729777e-21,  1.84952635e-06,  1.50807053e-22],
       [-9.97337310e-21,  1.50807053e-22,  1.04828946e-06]])

In [310]:
sigma1 = np.sqrt( np.dot(np.dot(np.transpose(dur_pca_1),covarianza),dur_pca_1) )
sigma2 = np.sqrt( np.dot(np.dot(np.transpose(dur_pca_2),covarianza),dur_pca_2) )

In [311]:
(VaR_CA := M1*sigma1*zq+M2*sigma2*zq )

2023660.9233775523


## c) Use ahora un enfoque de simulación histórica y las curvas provistas para analizar
el riesgo del portafolio.

In [315]:
n=len(cambios_tasas)
esceneario_1_precios = np.zeros(n)
esceneario_2_precios = np.zeros(n)
for i in range(0,n):
    esceneario_1_precios[i] = PriceTermStructShift(spread_estimado_1, cambios_en_tasas[i,:],bono_1)
    esceneario_2_precios[i] = PriceTermStructShift(spread_estimado_2, cambios_en_tasas[i,:],bono_2)

In [316]:
esceneario_1_precios

array([99.82662189, 99.83660687, 99.96510786, ..., 99.78351626,
       99.81850077, 99.95150933])

In [317]:
esceneario_2_precios

array([ 99.90330555,  99.94747637, 100.44553635, ...,  99.83632727,
       100.10393461, 100.37984557])

In [318]:
V_Hist = N1*esceneario_1_precios+N2*esceneario_2_precios

In [319]:
(VaR_Hist := (V-np.quantile(V_Hist,1-nivel_conf)) )

2490484.6527523696